In [1]:
# Bibliothèques standard
import os
import json
import re
import datetime

# Bibliothèques pour l'environnement
from dotenv import load_dotenv, dotenv_values

# Bibliothèques d'analyse de données
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliothèques de traitement de texte
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Bibliothèques de machine learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Bibliothèques Plaid
from plaid.api import plaid_api
from plaid.model.plaid_error import PlaidError
from plaid.model.products import Products
from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.country_code import CountryCode
from plaid.model.link_token_get_request import LinkTokenGetRequest
from plaid.model.accounts_get_request import AccountsGetRequest
from plaid.model.transactions_get_request import TransactionsGetRequest
from plaid.configuration import Configuration
from plaid.api_client import ApiClient
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest

In [ ]:
load_dotenv() 
env = 'sand'
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET_" + env)
PLAID_ENV = os.getenv("PLAID_ENV_" + env)
institutionId = "ins_1"

### plaid service


In [ ]:
def create_link_token():
    """
    Crée un token de liaison Plaid pour initialiser Plaid Link
    """
    client = get_plaid_client()
    
    request = LinkTokenCreateRequest(
        products=[Products('auth'), Products('transactions')],
        client_name="CashSense",
        country_codes=[CountryCode('US')],
        language='fr',
        user={'client_user_id': 'user_12345'}  # En production, utilisez un ID utilisateur réel
    )
    
    response = client.link_token_create(request)
    return response['link_token']

def exchange_public_token(public_token):
    """
    Échange un token public contre un token d'accès permanent
    """
    client = get_plaid_client()
    
    exchange_request = ItemPublicTokenExchangeRequest(
        public_token=public_token
    )
    exchange_response = client.item_public_token_exchange(exchange_request)
    
    # Dans une application réelle, stockez cet access_token en sécurité
    access_token = exchange_response.access_token
    item_id = exchange_response.item_id
    
    return {
        "access_token": access_token,
        "item_id": item_id
    }

def get_user_transactions(access_token, days=30):
    """
    Récupère les transactions des X derniers jours
    """
    client = get_plaid_client()
    
    # Calculer les dates de début et de fin
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=days)
    
    # Récupérer les transactions
    request = TransactionsGetRequest(
        access_token=access_token,
        start_date=start_date,
        end_date=end_date
    )
    
    response = client.transactions_get(request)
    transactions = response['transactions']
    
    # Formater les transactions pour le frontend
    return [format_transaction(transaction) for transaction in transactions]


### plaid helpers

In [ ]:
def get_plaid_client():
    """
    Configure et retourne un client Plaid
    """
    configuration = Configuration(
        host=PLAID_ENV,
        api_key={
            'clientId': PLAID_CLIENT_ID,
            'secret': PLAID_SECRET,
        }
    )
    api_client = plaid.ApiClient(configuration)
    return plaid_api.PlaidApi(api_client)

def format_transaction(transaction):
    """
    Formate une transaction Plaid pour le frontend
    """
    return {
        "id": transaction.transaction_id,
        "date": transaction.date,
        "name": transaction.name,
        "amount": transaction.amount,
        "category": ', '.join(transaction.category) if transaction.category else "Non catégorisé",
        "payment_channel": transaction.payment_channel,
        "pending": transaction.pending,
        "merchant_name": transaction.merchant_name
    }

### plaid routes

In [ ]:
@plaid_blueprint.route('/create_link_token', methods=['POST'])
def link_token_api():
    try:
        link_token = create_link_token()
        return jsonify({"link_token": link_token})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@plaid_blueprint.route('/exchange_token', methods=['POST'])
def exchange_token_api():
    try:
        public_token = request.json.get('public_token')
        if not public_token:
            return jsonify({"error": "Public token is required"}), 400
            
        result = exchange_public_token(public_token)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@plaid_blueprint.route('/get_transactions', methods=['POST'])
def transactions_api():
    try:
        access_token = request.json.get('access_token')
        if not access_token:
            return jsonify({"error": "Access token is required"}), 400
            
        transactions = get_user_transactions(access_token)
        return jsonify({"transactions": transactions})
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    
@plaid_blueprint.route('/create_sandbox_token', methods=['POST'])
def create_sandbox_token_api():
    try:
        institution_id = request.json.get('institution_id', 'ins_1')
        client = get_plaid_client()
        
        request_obj = SandboxPublicTokenCreateRequest(
            institution_id=institution_id,
            initial_products=[Products('transactions')]
        )
        response = client.sandbox_public_token_create(request_obj)
        return jsonify({"public_token": response.public_token})
    except Exception as e:
        return jsonify({"error": str(e)}), 500